# OPEN DATASET

In [1]:
import matplotlib
import numpy as np
import nmrglue as ng

import matplotlib.cm as cm
import matplotlib.pyplot as plt 
import matplotlib.colors as colors

import matplotlib.cbook as cbook
from matplotlib import cm
import matplotlib.image as mpimg

%matplotlib qt5

## 2D File
dic, data = ng.bruker.read_pdata('NA0048214682_3620_20221122_01/Bruker/3/pdata/1/') 
udic = ng.bruker.guess_udic(dic, data)

uc1 = ng.fileiobase.uc_from_udic(udic,1) ## Direct dimension
uc0 = ng.fileiobase.uc_from_udic(udic,0) ## indirect dimension

x0, x1 = uc1.ppm_limits()
y0, y1 = uc0.ppm_limits()
ppm_scale0 = uc0.ppm_scale()
ppm_scale1 = uc1.ppm_scale()
##print(x0,x1,y0,y1)
z=data
xppm,yppm=np.meshgrid(np.linspace(x0, x1, 2048), np.linspace(y0, y1, 512))
##

In [3]:
deltax = (x1-x0)/2048
deltay = (y1-y0)/512

x = np.arange(0, 2048, 1)#*deltax
y = np.arange(0, 512,  1)#*deltay

# difference of Gaussians
Z = data



# PLOT IMAGE

In [5]:
from bokeh.palettes import Sunset11, RdBu11, Iridescent, viridis, magma, Turbo256,RdYlBu
from bokeh.plotting import figure, show
from bokeh.models import LinearColorMapper, ColorBar, LogColorMapper

cl = data.std()/6 * 8 * 1.2 ** np.arange(8)

#cl3 = np.concatenate((-cl[::-1], cl))


color_mapper =LogColorMapper(palette="RdBu11", low=-cl[5], high=cl[5])

p = figure(x_range=(0, 2048), y_range=(0, 512),
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])

# must give a vector of image data for image parameter
#p.image(image=[Z], x=0, y=0,dw=2048, dh=512,  color_mapper=color_mapper)
#p.grid.grid_line_width = 0.5
# generate colorbar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0))

# add colorbar to the right side of the plot
#p.add_layout(color_bar, 'right')


#show(p)

In [22]:
import matplotlib as mpl

colorp = mpl.cm.Blues(np.linspace(0,1,40))
colorn = mpl.cm.Reds(np.linspace(0,1,40))
#print(colorn[19])
pmap=[]
nmap=[]
for i in range(39,19,-1):
    pcolor = mpl.colors.to_hex(colorp[i], keep_alpha=False)
    ncolor = mpl.colors.to_hex(colorn[19-i], keep_alpha=False)
    #print(ncolor)
    pmap.append(pcolor)
    nmap.append(ncolor)

#print(nmap)

# PLOT 2D CONTOUR 

In [25]:
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import figure, show,  output_file
import bokeh.palettes as palettes
from bokeh.models import CustomJS, Slider, RangeSlider, Div, Row
from bokeh.layouts import column, row
from bokeh.core.properties import value
output_file('index2d.html') ## used as final HTML file 

# Data to contour is the sum of two Gaussian functions.
title="Embrapa Copaiba Database Collection"

p = figure(width=1024, height=768, x_range=(x0, x1), y_range=(y0, y1),title= title)
cl = data.std()/9 * 8 * 1.2 ** np.arange(20)
cl3 = np.concatenate((-cl[::-1], cl))
levelsp = np.linspace(cl[7], cl[19], 20)
levels=levelsp
leveln=levelsp*-1
i = figure(x_range=(x0, x1), 
           y_range=(y0, y1),
           width=int(1024/4), 
           height=int(768/4))




contour_renderer = p.contour(x=xppm, y=yppm, z=z, levels=levels, fill_color="", line_color=pmap, line_width=2)

#colorbar = contour_renderer.construct_color_bar()
#p.add_layout(colorbar, "right")

contour_renderer = p.contour(x=xppm, y=yppm, z=z, levels=leveln[::-1], fill_color="",line_color=nmap, line_width=2)


#colorbar = contour_renderer.construct_color_bar()
#p.add_layout(colorbar, "right")
## Setup title
p.title.text_font = 'Courier'
p.title.text_font_size = '20pt'

## Setup Xaxis fonts
p.xaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font="Arial"
p.xaxis.major_label_text_font_size="14pt"
p.xaxis.axis_label_text_font = "Arial"
p.xaxis.axis_label = "$$^1H\ ppm$$"

## Setup yaxis fonts
p.yaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.major_label_text_font_size="14pt"
p.yaxis.axis_label_text_font = "Arial"
p.yaxis.axis_label = "$$^{13}C\ ppm$$"

url = "https://www.embrapa.br/documents/1355291/47137384/Logomarca+Embrapa.png/2999900a-8c32-96f5-8f73-1a591bba1ee1?t=1570557387741"
d1 = Div(text = '<div style="position: absolute; left:-265px; top:540px"><img src=' + 
         url + ' style="width:241px; height:171px; opacity: 1"></div>')

show(row(p, d1))
